Hi dear reviewer ! due to the size of model used (BERT is around 1Gb actually) we are using this notebook as a web application to present our model.
* **All you have to do is run the notebook entirely (click on "run everything")** [it takes about 15s to setup]
* **A link should appear at the bottom of the notebook (we are using [Dash](https://github.com/plotly/jupyter-dash) to make it possible 😉**)
* **click on it and ENJOY OUR WORK!**

# FIGHT GBV Web application

  ![](https://unwomen.org.au/wp-content/uploads/2020/11/Orange-the-world-banner.jpg)

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 1.6MB/s 
     |████████████████████████████████| 6.8MB 40.9MB/s 
     |████████████████████████████████| 983kB 39.3MB/s 
     |████████████████████████████████| 266kB 43.9MB/s 
     |████████████████████████████████| 1.9MB 36.7MB/s 
     |████████████████████████████████| 1.2MB 48.9MB/s 
     |████████████████████████████████| 471kB 50.7MB/s 
     |████████████████████████████████| 870kB 47.0MB/s 
     |████████████████████████████████| 3.2MB 36.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=f56a8ecb57bd6352c310ab3064e8686160ab2a2c8ca41e8f20ac51a3b99d3790
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=265da870f204d9397d892a70b826eb03fedb6c58116cfd9ea841b5f328ddf6a6
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import ktrain

In [ ]:
!pip install jupyter-dash

     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 1.0MB 33.6MB/s 
     |████████████████████████████████| 3.5MB 41.8MB/s 
     |████████████████████████████████| 194kB 47.6MB/s 
     |████████████████████████████████| 1.8MB 42.0MB/s 
     |████████████████████████████████| 358kB 44.8MB/s 
  Created wheel for dash: filename=dash-1.19.0-cp37-none-any.whl size=84011 sha256=a963e79316f2ec511efe830ca8d3a5b7d2b7d0c717752c42cfb2f2970a88fe89
  Stored in directory: /root/.cache/pip/wheels/f7/50/a7/a230ff7f503b10120bff18c2524a375bb85a61ce6b519c8a77
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-cp37-none-any.whl size=1014870 sha256=be267d6ba07153141880bfa23c16464bb46996c57fd00918ee041b39ef5d2113
  Stored in directory: /root/.cache/pip/wheels/46/a0/ec/2be2e8fc750e623d76f9690c397cc5ab28b33d0a16a49e10c5
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-cp37-none-any.whl size=3527014 sha256=f5ee4c5bb01a913f8104

## Downloading the model (Way to heavy [999M] to be downloaded locally )

In [ ]:
import re
def remove_users(x):
  return re.sub(r"@\w+",' @user',x)

def remove_all_nonalphanumeric(text):
  return re.sub(r"[^a-z'À-ÖØ!?-öø-ÿ,]+", ' ', text)

def remove_hashtag(x):
  return re.sub(r"#\w+","",x)

def remove_hyperlinks(text):
  text=re.sub(r'http\S+', '', text, flags=re.MULTILINE)
  return re.sub(r'www\S+', '', text, flags=re.MULTILINE)

In [ ]:
def preprocess(s):
  s=s.lower()
  s=remove_hashtag(s)
  s=remove_users(s)
  s=remove_hyperlinks(s)
  s=remove_all_nonalphanumeric(s)
  return s

In [ ]:
import requests
import os

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [ ]:
!mkdir sexism_classifier

In [ ]:
download_file_from_google_drive('1-0hFZUA2ZcoR6Pn_MRVBkSnSQjs7dbvA','sexism_classifier/config.json')

In [ ]:
download_file_from_google_drive('1-57sNgARicmhteb9W4co8AKm1oAl8JzA','sexism_classifier/tf_model.h5')

In [ ]:
download_file_from_google_drive('1-Bm0mxz_KR3pImHW74qgAt1_S0pmrjAV','sexism_classifier/tf_model.preproc')

In [ ]:
predictor=ktrain.load_predictor('sexism_classifier')

## Setting up application

In [ ]:
# imports
from jupyter_dash import JupyterDash
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
from dash import no_update
import base64
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1(children="Gender-Based Violence Detection", style={'textAlign': 'center'}),
    dcc.Markdown('''
                ![](https://teentalk.ca/wp-content/uploads/2020/11/ened-gbv-702x330.jpg)
                ###### Step 1: Type a sentence (a short one preferably **et en FRANÇAIS**).
                ###### Step 2: Click submit
                ###### Step 3: Wait for your prediction to appear!

                example: "va à la cuisine" (go to kitchen) vs "femme vas à la cuisine" (women go to kitchen)
                
    ''',style={'textAlign': 'center'}),
    
    dcc.Input(id='username',type='text',style={'height':'50px','width':'250px'}),
    html.Button(id='submit-button', type='submit', children='Submit'),
    html.Div(id='output_div')
    ],style={'textAlign': 'center','justify':'center','align':'middle','verticalAlign':'middle'})

@app.callback(Output('output_div', 'children'),
                [Input('submit-button', 'n_clicks')],
                [State('username', 'value')],
                )
def update_output(clicks, input_value):
    if clicks is not None:
        if input_value is not None:
          if len(input_value.split(' '))<70:
            answ = predictor.predict([preprocess(input_value)])
            answ_proba = predictor.predict_proba([input_value])
            answ_proba ={
                'DOUBTFUL':answ_proba[0][0],
                'NON-SEXIST':answ_proba[0][1],
                'SEXIST':answ_proba[0][2]
            }
            return(html.Div([html.H2(children=str(answ[0]), style={'textAlign': 'center'}),
                            html.H4(children=str(answ_proba), style={'textAlign': 'center'})
                    ]))
          return("The sentence is too long!")
                
        return('Please enter a valid sentence')
           


# APPLICATION 👇🐱‍🏍

In [ ]:
# After running all the notebook, a link should appear just below, just click on it
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

This app is a demonstration of how machine learning can help make the internet a better place for women who experience free violence on social media. Therefore, like the embryonic bots set up by these large companies to detect hate speech, we offer the possibility of spotting and reporting gender-based violence speech. Because they are our mothers, our sisters and our wives, we must protect them, together we will succeed.

Learn more about GBV in Africa with those links :

* [Online Gender Violence affects 45% of women on social media in west and central-africa](https://internetwithoutborders.org/iwd2019-online-gender-based-violence-affects-45-of-women-on-social-media-in-west-and-central-africa/)
* [GBV prevention in Africa](https://preventgbvafrica.org/)
* [The silent epidemic](https://blogs.worldbank.org/africacan/silent-epidemic)
* [Hate on social network in Cameroon](https://defyhatenow.org/wp-content/uploads/2020/09/1_dhn-Cameroon_FG_FR_FINAL_ONLINE.pdf)